In [1]:
import requests
import json 
import pickle
import os 
import pandas as pd 
import json
import time
import math 
import numpy as np

In [2]:
headers_myapp = {
    'API-Key': 'XXX' ,
    'Accept': 'application/hal+json'
}
headers_myapp2 = {
    'API-Key': 'XXX' ,
    'Accept': 'application/hal+json'
}
headers_extract1 = {
    'API-Key': 'XX' ,
    'Accept': 'application/hal+json'
}

Set up extraction follow up file 

In [ ]:
# Appels pour avoir le nombre de pages par jour sur le mois de janvier
import datetime
startrange_ = datetime.datetime(2026, 1, 1)
numdays = 32
date_list = [startrange_ + datetime.timedelta(days=x) for x in range(numdays)]
date_list = [sub.strftime("%Y-%m-%dT%H:%M:%S.%fZ") for sub in date_list]

totalPagesPerDay_ = []
for i in range(len(date_list)-1):
    #if i not in [0,1]:
        try: 
            print(date_list[i])
            print(date_list[i+1])

            startrange_ = date_list[i]
            endrange_ = date_list[i+1]
            #endrange_ = "2026-01-01T08%3A00%3A00.000Z"

            # Premier appel pour avoir le nombre de pages 
            response = requests.get(url=f"https://api.airfranceklm.com/opendata/flightstatus?startRange={startrange_}&endRange={endrange_}", headers=headers)
            print(response.status_code)
            totalPages = json.loads(response.content.decode('utf-8'))["page"]['totalPages'] # Nombre total de pages 
            totalPagesPerDay_.append(totalPages)
            time.sleep(5)
        except:
            print(f"Call {startrange_} to {endrange_} failed with error {response.status_code}")
            totalPagesPerDay_.append("failed")
            time.sleep(5)

# Create DF
totalPagesPerDay_df = pd.DataFrame({"from" : date_list[0:len(date_list)-1],
                                    "to" : date_list[1:len(date_list)],
                                    "totalPages" : totalPagesPerDay_},
                                    "")

# Add necessary columns for extraction 
totalPagesPerDay_df['lastExtractedPage'] = np.nan
worker_array = ["myapp","myapp2","extract1"]
totalPagesPerDay_df['worker'] = np.repeat(worker_array, math.ceil(totalPagesPerDay_df.shape[0]/3))[0:totalPagesPerDay_df.shape[0]]
totalPagesPerDay_df['complete'] = np.repeat(False, totalPagesPerDay_df.shape[0])

# Save DF
totalPagesPerDay_df.to_csv("/home/pierre/Documents/DE_DATASCIENTEST/dst_airlines/api/AFKLM_totalPagesPerDay_df_jan26.csv")
sum(totalPagesPerDay_df.loc[totalPagesPerDay_df.totalPages != "failed"]["totalPages"].apply(lambda x:int(x))) * 100  # taille espérée de la base de données 


In [4]:
totalPagesPerDay_df = pd.read_csv("/home/pierre/Documents/DE_DATASCIENTEST/dst_airlines/api/AFKLM_totalPagesPerDay_df_jan26.csv", index_col=0)
totalPagesPerDay_df.head()

,from,to,totalPages,lastExtractedPage,worker,complete
0,2026-01-01T00:00:00.000000Z,2026-01-02T00:00:00.000000Z,87,NaN,myapp,False
1,2026-01-02T00:00:00.000000Z,2026-01-03T00:00:00.000000Z,94,NaN,myapp,False
2,2026-01-03T00:00:00.000000Z,2026-01-04T00:00:00.000000Z,failed,NaN,myapp,False
3,2026-01-04T00:00:00.000000Z,2026-01-05T00:00:00.000000Z,failed,NaN,myapp,False
4,2026-01-05T00:00:00.000000Z,2026-01-06T00:00:00.000000Z,failed,NaN,myapp,False


Set up extraction

In [ ]:
headers_attribution = {"myapp":headers_myapp,
                       "myapp2":headers_myapp2,
                       "extract1":headers_extract1}


for worker_ in set(totalPagesPerDay_df['worker']):
    print(worker_)
    # Remaining work 
    sub_df = totalPagesPerDay_df.loc[(totalPagesPerDay_df['worker'] == worker_) & (-totalPagesPerDay_df['complete']) & (totalPagesPerDay_df['totalPages']!='failed')]
    #  
    for row_ in range(sub_df.shape[0]):
        from_, to_, tp_, lep_ = sub_df.iloc[row_][["from", "to","totalPages", "lastExtractedPage"]]
            
        print(from_)
        if math.isnan(lep_):
            start_page_ = 0
        else:
            start_page_ = int(lep_) + 1 
        # Appel
        for page_ in range(start_page_+1, int(tp_)):
            try:
                response = requests.get(url=f"https://api.airfranceklm.com/opendata/flightstatus?startRange={from_}&endRange={to_}&pageNumber={page_}", headers=headers_attribution[worker_])
                with open(f'/home/pierre/Documents/DE_DATASCIENTEST/dst_airlines/data/afklm_large/AFKLM_get_flight_{from_[0:10].replace("-","_")}_{to_[0:10].replace("-","_")}_page_{page_}.json', 'w') as f:
                    json.dump(response.json(), f, indent=4)
                
            except:
                print(f"Call page {page_} failed with error {response.status_code}")
                if response.status_code == 403:
                    totalPagesPerDay_df = pd.read_csv("/home/pierre/Documents/DE_DATASCIENTEST/dst_airlines/api/AFKLM_totalPagesPerDay_df_jan26.csv", index_col=0)
                    if start_page_ == page_:
                        if start_page_ == 0 :
                            lep_update = np.nan
                        else:
                            lep_update = page_ - 1
                    else:
                        lep_update = page_ - 1 
                    totalPagesPerDay_df.loc[totalPagesPerDay_df["from"] == from_,'lastExtractedPage']= lep_update
                    totalPagesPerDay_df.to_csv("/home/pierre/Documents/DE_DATASCIENTEST/dst_airlines/api/AFKLM_totalPagesPerDay_df_jan26.csv")
                    break


            if page_ == tp_:
                totalPagesPerDay_df = pd.read_csv("/home/pierre/Documents/DE_DATASCIENTEST/dst_airlines/api/AFKLM_totalPagesPerDay_df_jan26.csv", index_col=0)
                totalPagesPerDay_df.loc[totalPagesPerDay_df["from"] == from_,"lastExtractedPage"] = page_
                totalPagesPerDay_df.loc[totalPagesPerDay_df["from"] == from_,"complete"] = True
                totalPagesPerDay_df.to_csv("/home/pierre/Documents/DE_DATASCIENTEST/dst_airlines/api/AFKLM_totalPagesPerDay_df_jan26.csv")
                print(f"{from_} extrait.")
            time.sleep(5)



    



myapp2
2026-01-12T00:00:00.000000Z
